<a href="https://colab.research.google.com/github/satyamnewale/basic-NLP/blob/main/batching(final-model-full-execute).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
from sklearn.datasets import make_classification
import torch

In [102]:
#Step 1. create a synthetic dataset using sklearn

X,y = make_classification(
    n_samples = 10,
    n_features = 2,
    n_classes = 2,
    n_informative=2,
    n_redundant=0,
    random_state=42
)

In [103]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [104]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [105]:
#convert to tensor
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [106]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [107]:
from torch.utils.data import Dataset,DataLoader

In [108]:
#sequential
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = features # store
    self.labels = labels # store

  def __len__(self):
    return self.features.shape[0] # returns features index

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx] # returns features and labels

In [109]:
dataset = CustomDataset(X,y)

In [110]:
len(dataset)

10

In [111]:
dataLoader = DataLoader(dataset, batch_size=2, shuffle=True, num_workers=2, pin_memory=True)

In [112]:
for batch_features, batch_labels in dataLoader:

  print(batch_features)
  print(batch_labels)
  print("-"*50)

tensor([[-2.8954,  1.9769],
        [ 1.7273, -1.1858]])
tensor([0, 1])
--------------------------------------------------
tensor([[-0.7206, -0.9606],
        [-0.9382, -0.5430]])
tensor([0, 1])
--------------------------------------------------
tensor([[-0.5872, -1.9717],
        [ 1.8997,  0.8344]])
tensor([0, 1])
--------------------------------------------------
tensor([[-1.9629, -0.9923],
        [-1.1402, -0.8388]])
tensor([0, 0])
--------------------------------------------------
tensor([[ 1.7774,  1.5116],
        [ 1.0683, -0.9701]])
tensor([1, 1])
--------------------------------------------------


---

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)
df.head()


,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
df.shape

(569, 31)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size=0.2)

In [ ]:
#scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

array([[-1.84410785,  1.42396119, -1.81364149, ..., -0.97923111,
        -0.16567969,  1.26977724],
       [ 0.39925491,  0.02635142,  0.42322432, ...,  0.58905694,
        -0.08968423, -0.19919309],
       [-0.67492574, -1.2540092 , -0.7279064 , ..., -0.47038815,
         0.33718386, -0.93951853],
       ...,
       [ 1.14136359, -0.59037906,  1.08556465, ...,  0.62153628,
        -0.08159961, -0.08461229],
       [-1.20508587, -0.30663614, -1.13201784, ..., -0.21983325,
        -0.45025842,  1.803746  ],
       [-0.82508003,  0.11546077, -0.82432303, ..., -1.43981906,
         0.64601646, -1.03185063]])

In [ ]:
#label encoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
y_train

array([0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1,

In [ ]:
#numpy to pytorch tensors
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train.astype(np.float64))
y_test_tensor = torch.from_numpy(y_test.astype(np.float64))

In [ ]:
X_train_tensor

tensor([[-1.8441,  1.4240, -1.8136,  ..., -0.9792, -0.1657,  1.2698],
        [ 0.3993,  0.0264,  0.4232,  ...,  0.5891, -0.0897, -0.1992],
        [-0.6749, -1.2540, -0.7279,  ..., -0.4704,  0.3372, -0.9395],
        ...,
        [ 1.1414, -0.5904,  1.0856,  ...,  0.6215, -0.0816, -0.0846],
        [-1.2051, -0.3066, -1.1320,  ..., -0.2198, -0.4503,  1.8037],
        [-0.8251,  0.1155, -0.8243,  ..., -1.4398,  0.6460, -1.0319]],
       dtype=torch.float64)

In [ ]:
loss_function = nn.BCELoss()

In [ ]:
from sklearn.datasets import make_classification
from torch.utils.data import Dataset,DataLoader

#sequential
class CustomDataset(Dataset):

  def __init__(self, features, labels):
    self.features = features # store
    self.labels = labels # store

  def __len__(self):
    return len(self.features)

  def __getitem__(self, idx):
    return self.features[idx], self.labels[idx] # returns features and labels

In [ ]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [ ]:
train_Loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2,  pin_memory=True)
test_Loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2, pin_memory=True )

In [ ]:
# #defining the model

# class MySimpleNN():

#   def __init__(self, X):
#     self.weights = torch.rand(X.shape[1],1, dtype = torch.float64, requires_grad=True)
#     self.bias = torch.zeros(1, dtype = torch.float64, requires_grad=True)

#   def forward(self, X):
#     z = torch.matmul(X, self.weights) + self.bias
#     y_pred = torch.sigmoid(z)
#     return y_pred

#   def loss_function(self, y_pred, y):
#     ## clamps prediction to avoid log(0) and log(1)
#     epsilon = 1e-7
#     y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
#     ## calculate loss
#     loss = (-(y*torch.log(y_pred) + (1-y)*torch.log(1-y_pred))).mean()
#     return loss

class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, X):
    out = self.linear(X)
    out = self.sigmoid(out)
    return out

  # def loss_function(self, y_pred, y):
  #   ## clamps prediction to avoid log(0) and log(1)
  #   epsilon = 1e-7
  #   y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
  #   ## calculate loss
  #   loss = (-(y*torch.log(y_pred) + (1-y)*torch.log(1-y_pred))).mean()
  #   return loss

In [ ]:
#important parameters

learning_rate = 0.05
epochs = 501

In [ ]:
X_train_tensor.shape[1]

30

In [ ]:
#training pipeline
## create model
model = MySimpleNN(X_train_tensor.shape[1])
model = model.double() # Convert model parameters to double

#define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## loop
for epoch in range(epochs):
  for batch_features,batch_labels in train_Loader:

      ## forward pass
      y_pred = model(batch_features) # Use X_train_tensor directly

      ## loss
      loss = loss_function(y_pred, batch_labels.view(-1,1))

      ## zero gradient
      optimizer.zero_grad()

      ## backward pass
      loss.backward()

      #now update weights and bias
      # with torch.no_grad():
      #   model.linear.weight -= learning_rate * model.linear.weight.grad
      #   model.linear.bias -= learning_rate * model.linear.bias.grad
      optimizer.step()

      #zero gradient
      # model.linear.weight.grad.zero_()
      # model.linear.bias.grad.zero_()


      #print loss
      if epoch % 100 == 0:
        print(f'Epoch: {epoch}, Loss: {loss.item()}')


Epoch: 0, Loss: 0.6822054138039071
Epoch: 0, Loss: 0.5870060894506856
Epoch: 0, Loss: 0.5744138752579884
Epoch: 0, Loss: 0.5116194665507281
Epoch: 0, Loss: 0.4994305678572482
Epoch: 0, Loss: 0.4216770230396091
Epoch: 0, Loss: 0.38582673560061076
Epoch: 0, Loss: 0.4144291364320625
Epoch: 0, Loss: 0.38433267160580376
Epoch: 0, Loss: 0.326232882253883
Epoch: 0, Loss: 0.3389794462461255
Epoch: 0, Loss: 0.31420097745438147
Epoch: 0, Loss: 0.2707389654137051
Epoch: 0, Loss: 0.26963577168057995
Epoch: 0, Loss: 0.4193809845801141
Epoch: 100, Loss: 0.03985227741293544
Epoch: 100, Loss: 0.014860030117550577
Epoch: 100, Loss: 0.16189495991545744
Epoch: 100, Loss: 0.08981503654385359
Epoch: 100, Loss: 0.04281094819091394
Epoch: 100, Loss: 0.06259353054185597
Epoch: 100, Loss: 0.05364278497255415
Epoch: 100, Loss: 0.1093295550721604
Epoch: 100, Loss: 0.0169647855396443
Epoch: 100, Loss: 0.030889925837751304
Epoch: 100, Loss: 0.036316114513977
Epoch: 100, Loss: 0.058693109562893184
Epoch: 100, Loss:

In [ ]:
## check loss , weight and bias

print(f"Loss: {loss.item()}, weight: {model.linear.weight}, bias: {model.linear.bias.item()}")

Loss: 0.0028760312851391426, weight: Parameter containing:
tensor([[ 0.5528,  0.6165,  0.3750,  0.4987,  0.1958, -0.7821,  0.9698,  1.2075,
         -0.1333, -0.1404,  1.5345, -0.1023,  1.0024,  1.0246,  0.3147, -1.1205,
          0.0526,  0.3832, -0.8245, -0.7793,  1.1362,  1.1186,  1.2155,  1.1326,
          0.9478, -0.2075,  1.1573,  0.9834,  0.8331,  0.6760]],
       dtype=torch.float64, requires_grad=True), bias: -0.49572919808271404


In [ ]:
#model evaluation
model.eval()
accuracy = []
with torch.no_grad():
  for batch_features,batch_labels in test_Loader:
      #forward pass
      y_pred = model(batch_features)
      y_pred = (y_pred > 0.5).float()

      #calculate accuracy for current batch
      batch_acc = (y_pred.view(-1) == batch_labels).float().mean().item()
      accuracy.append(batch_acc)

#calculate overall accuracy
overall_acc = sum(accuracy)/len(accuracy)
print(f'Overall Accuracy: {overall_acc}')

## with torch.no_grad():
#   y_pred = model.forward(X_test_tensor)
#   y_pred = (y_pred > 0.9).float()
#   accuracy = (y_pred == y_test_tensor).float().mean()
#   print(f'Accuracy: {accuracy.item()}')

Overall Accuracy: 0.96875
